In [5]:
import os
import zipfile
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

In [6]:
download_path = os.getcwd() + "\\downloads"

options = webdriver.ChromeOptions()
prefs = {"download.default_directory" : download_path}
options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(options=options)
driver.maximize_window()

url = "https://plvr.land.moi.gov.tw/DownloadOpenData"
driver.get(url)

parent = driver.current_window_handle
uselessWindows = driver.window_handles
driver.switch_to.window(uselessWindows[-1])
driver.close()
driver.switch_to.window(uselessWindows[0])

In [7]:
element_history = driver.find_element(by=By.LINK_TEXT, value="非本期下載")
element_history.click()
sleep(5)

element_format = driver.find_element(by=By.ID, value="fileFormatId")
select = Select(element_format)
select.select_by_value("csv")

element_season = driver.find_element(by=By.ID, value="historySeason_id")
list_historySeason = element_season.text.split()
select = Select(element_season)

for historySeason in list_historySeason:
    select.select_by_visible_text(historySeason)

    element_download = driver.find_element(by=By.ID, value="downloadBtnId")
    element_download.click()

    while not os.path.exists("./downloads/lvr_landcsv.zip"):
        sleep(1)

    with zipfile.ZipFile("./downloads/lvr_landcsv.zip", 'r') as zip_ref:
        zip_ref.extractall(f"./data/{historySeason}/")

    os.remove("./downloads/lvr_landcsv.zip")

driver.close()